In [2]:
import os
import librosa
import numpy as np
import pandas as pd
import joblib
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Set configs
sr = 32000
chunk_duration = 5  # seconds
n_mels = 128
perc_threshold = 90.0

# Paths
MODEL_PATH = "/kaggle/input/206-bird-detector/birdclef_models.joblib"
AUDIO_BASE_TEST = "/kaggle/input/birdclef-2025/test_soundscapes/"
AUDIO_BASE_TRAIN = "/kaggle/input/birdclef-2025/train_audio/"

# Load class labels
class_labels = sorted(os.listdir(AUDIO_BASE_TRAIN))

# Load models
print("📦 Loading trained models...")
models = joblib.load(MODEL_PATH)
print(f"✅ Loaded {len(models)} species models")

# Get test soundscape files
try:
    test_files = sorted([f for f in os.listdir(AUDIO_BASE_TEST) if f.endswith(".ogg")])
except:
    test_files = []

# If test files not visible (e.g. local run), simulate with train audio
if len(test_files) == 0:
    print("⚠️ No test files found. Simulating with 3 train samples...")
    test_files = []
    for bird in class_labels[:3]:
        species_dir = os.path.join(AUDIO_BASE_TRAIN, bird)
        for f in os.listdir(species_dir):
            if f.endswith(".ogg") or f.endswith(".mp3"):
                test_files.append(os.path.join(species_dir, f))
                break
else:
    test_files = [os.path.join(AUDIO_BASE_TEST, f) for f in test_files]

# Feature extraction
def extract_chunk_features(y):
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, power=2.0)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    band_means = mel_db.mean(axis=1)
    band_stds = mel_db.std(axis=1)
    thresh = np.percentile(mel, perc_threshold)
    mask = (mel > thresh)
    frac_active = mask.sum(axis=1) / mask.shape[1]
    return np.concatenate([band_means, band_stds, frac_active])

# Inference
submission_rows = []

for file_path in tqdm(test_files, desc="🔊 Running inference"):
    try:
        y, _ = librosa.load(file_path, sr=sr)
    except Exception as e:
        print(f"❌ Failed to load {file_path}: {e}")
        continue

    samples_per_chunk = sr * chunk_duration
    num_chunks = len(y) // samples_per_chunk

    for i in range(num_chunks):
        chunk = y[i*samples_per_chunk : (i+1)*samples_per_chunk]
        if len(chunk) < samples_per_chunk:
            continue

        try:
            feat = extract_chunk_features(chunk)
        except Exception as e:
            print(f"❌ Feature extraction failed: {e}")
            continue

        # Build row_id
        base_name = os.path.basename(file_path).replace(".ogg", "").replace(".mp3", "")
        row_id = base_name + f"_{(i+1)*chunk_duration}"
        row = [row_id]

        # Predict for all 206 birds
        for bird in class_labels:
            model = models.get(bird)
            try:
                prob = model.predict_proba(feat.reshape(1, -1))[:, 1][0] if model else 0.001
            except Exception:
                prob = 0.001
            row.append(prob)

        submission_rows.append(row)

# Final dataframe
submission_df = pd.DataFrame(submission_rows, columns=["row_id"] + class_labels)
submission_df.to_csv("submission.csv", index=False)
print("✅ submission.csv saved!")

📦 Loading trained models...
✅ Loaded 206 species models
⚠️ No test files found. Simulating with 3 train samples...


🔊 Running inference: 100%|██████████| 3/3 [00:14<00:00,  4.70s/it]

✅ submission.csv saved!
